In [3]:
from faker import Faker
import numpy as np
import pandas as pd
import boto3
import psycopg2

In [4]:
# Database connection parameters
db_name = 'factihealth'   # Database name
db_user = 'fh_user'  # Username
db_password = 'Facti@874'  # Password
db_host = 'redshift-cluster-factihealth.cuzgotkwtow6.ap-south-1.redshift.amazonaws.com'  # Cluster endpoint
db_port = 5439  # Port
# Connect to the database
try:b
    conn = psycopg2.connect(
        dbname=db_name,
        user=db_user,
        password=db_password,
        host=db_host,
        port=db_port
    )
    print("Connected to the database successfully")
    # Create a cursor object
    cur = conn.cursor()
    # Execute a query
    cur.execute("select * from factihealth.mimic.patients")
    # Fetch the result
    rows = cur.fetchall()

    # Get the column names
    column_names = [desc[0] for desc in cur.description]

    # Create a DataFrame
    master_data = pd.DataFrame(rows, columns=column_names)

    print('Dataframe shape:', master_data.shape)
    # Fetch and print the result
except Exception as e:
    print(f"Database connection failed due to {e}")

Connected to the database successfully
Dataframe shape: (299712, 6)


In [5]:
master_data

,subject_id,gender,anchor_age,anchor_year,anchor_year_group,dod
0,10000032,F,52,2180,2014 - 2016,2180-09-09
1,10000084,M,72,2160,2017 - 2019,2161-02-13
2,10000102,F,27,2136,2008 - 2010,None
3,10000115,M,24,2154,2017 - 2019,None
4,10000117,F,48,2174,2008 - 2010,None
...,...,...,...,...,...,...
299707,19999376,M,44,2145,2011 - 2013,None
299708,19999442,M,41,2146,2008 - 2010,None
299709,19999464,F,35,2160,2008 - 2010,None
299710,19999750,M,45,2144,2017 - 2019,None


In [14]:
master_data['gender'].value_counts()

F    158553
M    141159
Name: gender, dtype: int64

In [15]:
# Let's generate unique names for each subject_id based on gender.

# First, we'll simulate a DataFrame with 'subject_id' and 'gender' for demonstration purposes.
# In a real scenario, you would load this DataFrame from your existing data.
import pandas as pd

# Example DataFrame
df = master_data[['subject_id','gender']]

# Now, generate unique names for each gender
fake = Faker()

# We use set to avoid duplicate names
male_names = set()
female_names = set()

# Assuming we have an equal number of males and females
# We will generate names until we have enough for each gender in the dataframe
while len(male_names) < df['gender'].value_counts()['M']:
    male_names.add(fake.name_male())

while len(female_names) < df['gender'].value_counts()['F']:
    female_names.add(fake.name_female())

# Convert the sets to lists so we can iterate over them
male_names = list(male_names)
female_names = list(female_names)

print(males_names)
print(female_names)

# Assign a unique name to each subject_id based on gender
def assign_name(row):
    if row['gender'] == 'M':
        return male_names.pop()
    elif row['gender'] == 'F':
        return female_names.pop()

# Apply the function to the DataFrame
df['name'] = df.apply(assign_name, axis=1)

df.head()  # Display the first few rows of the dataframe with the names


KeyboardInterrupt: 

In [18]:
import  random
df =  master_data[['subject_id','gender']]

# Generate names
# Assuming the maximum number of names needed for any gender is known
num_names = max(df['gender'].value_counts())

# Create lists of male and female names
male_names = [fake.first_name_male() for _ in range(num_names)]
female_names = [fake.first_name_female() for _ in range(num_names)]

# Shuffle the lists to randomize name assignment
random.shuffle(male_names)
random.shuffle(female_names)

# Function to assign names based on gender
def assign_name(row, male_names, female_names):
    if row['gender'] == 'M':
        return male_names.pop()
    else:  # Assuming only 'M' and 'F' are present in the gender column
        return female_names.pop()

# Apply the function to assign names
df['name'] = df.apply(assign_name, axis=1, male_names=male_names, female_names=female_names)

# Display the first few rows of the dataframe
df.head()


C:\Users\krishika.R\AppData\Local\Temp\ipykernel_32596\985510730.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['name'] = df.apply(assign_name, axis=1, male_names=male_names, female_names=female_names)


,subject_id,gender,name
0,10000032,F,Jill
1,10000084,M,Oscar
2,10000102,F,Shannon
3,10000115,M,Frank
4,10000117,F,Crystal


In [19]:
df.to_csv('fake_names.csv')